In [567]:
!pip install beautifulsoup4

In [568]:
!pip install lxml

In [569]:
!pip install request

In [570]:
from bs4 import BeautifulSoup
import requests

<h3>Read in the first table from the html page and display the first 10 rows.</h3>

In [571]:
import pandas as pd
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<h3>Drop all rows where the borough is not assigned.</h3>

In [572]:
index_names = table[table['Borough'] == 'Not assigned'].index
table.drop(index_names, axis=0, inplace=True)
table.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


<h3>At this point, no borough has a 'not assigned' designation. Therefore, we can move onto identifying the rows without a neighbourhood designation and assign it with their respective borough name.</h3>

In [573]:
index_names1 = table[table['Neighbourhood'] == 'Not assigned'].index
index_names1

Int64Index([8], dtype='int64')

<h3>We can now replace all neighbourhood with designation 'not assigned' with their respective borough names.</h3>

In [574]:
table['Neighbourhood'].replace('Not assigned',table['Borough'], inplace=True)
table.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


<h3>Groupby Postcode and Borough in order to compile common neighbourhoods into a single cell to match the shared postcode.</h3>

In [575]:
table = table.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list).to_frame().reset_index()
table.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


<h3>Table dimensions.</h3>

In [576]:
table.shape

(103, 3)

<h3>Derive the latitude and longitude of each neighborhood</h3>

In [577]:
!pip install geocoder
import geocoder

In [578]:
table1 = table['Postcode'] + ', Toronto, Ontario'
address_table = pd.DataFrame(table1).rename(columns = {'Postcode':'Address'})
address_table.head()

,Address
0,"M1B, Toronto, Ontario"
1,"M1C, Toronto, Ontario"
2,"M1E, Toronto, Ontario"
3,"M1G, Toronto, Ontario"
4,"M1H, Toronto, Ontario"


In [579]:

list1 = address_table['Address'].to_list()
for address in list1:
    geo_info = geocoder.google(address)
    coordinates = geo_info.latlng


<h3>Geocoder was not returning coordinates, so the csv file was used.</h3>

In [581]:
path = 'https://public.boxcloud.com/d/1/b1!dthtHkQ8CpGFpLqEMgNPV1N2yRaaY3bksjgx0LSOJoWbHMuqWS0YfbEEmD_zNkj-a-FKUflIG1ktK8tzflhT5gtnG0Tf1S-uJHx1zghZB7-mCq005MSL-GIQCFmsR4uM4a570MuGk1heYC0jyTieO3AfSi9lmPcnUJXKPuo-ZM76snGDwbB_InjRcgRLlChUWrGE45QU8TD2D1ulQ19zksA9qzTn21OWmRKzSMghDoIaeJHhpe4wjyjCf_sFCyzyiBW6qV5g14BdqaxRYfI1j73bHCTmZr02G0UNAx4YB8Z0Oxio4R9ZIvbC7HCZZLrrzDdSEODk1o_ruB9EUW4lWh5DL6YQOnv-lYBqKHxLjZL36tdbTRtKTyRYaaT5ObVvbhJ4tXclIx1pEPsQTwDARvY9PfJU1yO9WUdsjBqsypst7iNm9soB2varHDATJy6Wi_JoKmvHoMu_6Ls19kiA8i3_WaR2fmReyUzny1y1zrN22DrO096kIErH5xbL9iEuLqt9mPIH5dCERRIIaVIx7wdlTXkwUjH1qH6_hQ8gfpr4Hs-BS7MUpJh8MXRe2H-hOPQM5A-mLhnKMl30SjUz777C7xeOjG1AchlDaf9TJeDK_XSVlzUsIYlRD72jsgY8HhPmMXpCcANtzcsPQQdJt2xFOPgejTS7xfOE-KtJmRapnaMoZ5iHp3Qo08C3oKcKa8y9dbcqda7hzMuDgrS8yDEr96wuaKo1pT3gRbk03v5kG299s6lW039JLi5IeICyKJAUmoYEhbjWkjvNayddytoIuOgPVFqdqvAOfBU-vh0xvdMmNOE4sEIlEpoVTkDUswQGJ8a5vD8pyk1XdkmunlUs6uaz9AGDDGZajcnN6ABtzu2COVO4QNBklgI_IVKeEd4L55-W2Iy1ohdKlaLAiocFEUX6uRjcdszJ1giGy47ihdFxaRhwlZ1rCSwXxODQwmoCZdPSGoMAptBXtWf_DB0OQZeI1HJ54UgYBABA3YMftRKrm_JZbTNtxrwggS3U5MTg5Bi_sWCJfob1RdKEl7hW6rrnZXlIf1rMyLvPVxBQnr7Hw_s0x8nUKXii7UwsRkLvAs0W4kYZo8HL-daN_ZqZYjT5rpbNIrAHp6jo3chrX1cBgqbuTQ3jl0zEvYzb_y7b5C0x7SgQS18fabL7vCBrqMd-uY8w28GZFG-Pxs5ud-GfialOxEFRbVo07sKK-0yNK8cpJBU9A-x50s4kiHl9-Ewi3VTUW43waYWWKYFi7XWiU3ni99_MOtcSFadO0jdrZyf9gVrXcrupxQRItgJQh5kavJaf_2hfBakgvfTRpxi7G6B4dgyuRvE5ZDpcDXIM80W5D9sUXY4KtGY5v5IUHhmnOP56l0XV1uWG67f_xj0-tiog98fDdA../download'
df = pd.read_csv(path)
df.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


<h3>As the postal code is in alphabetical order (as is the 'table' dataframe), we can now compile postal codes with their respective neighbourhoods, boroughs, and coordinates.</h3>

In [669]:
df_final = pd.concat([table, df['Latitude'], df['Longitude']],axis=1)
df_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


<h3>Render the df more concise by only including neighbourhoods within the borough of Toronto.</h3>

In [583]:
WT = df_final[df_final['Borough'] == 'West Toronto']
ET = df_final[df_final['Borough'] == 'East Toronto']
CT = df_final[df_final['Borough'] == 'Central Toronto']
DT = df_final[df_final['Borough'] == 'Downtown Toronto']
toronto_addresses = pd.concat([WT,ET,CT,DT], axis=0).reset_index(drop=True)
toronto_addresses

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6H,West Toronto,"[Dovercourt Village, Dufferin]",43.669005,-79.442259
1,M6J,West Toronto,"[Little Portugal, Trinity]",43.647927,-79.419750
2,M6K,West Toronto,"[Brockton, Exhibition Place, Parkdale Village]",43.636847,-79.428191
3,M6P,West Toronto,"[High Park, The Junction South]",43.661608,-79.464763
4,M6R,West Toronto,"[Parkdale, Roncesvalles]",43.648960,-79.456325
5,M6S,West Toronto,"[Runnymede, Swansea]",43.651571,-79.484450
6,M4E,East Toronto,[The Beaches],43.676357,-79.293031
7,M4K,East Toronto,"[The Danforth West, Riverdale]",43.679557,-79.352188
8,M4L,East Toronto,"[The Beaches West, India Bazaar]",43.668999,-79.315572
9,M4M,East Toronto,[Studio District],43.659526,-79.340923


<h3>Convert the column 'Neighbourhood' from lists to strings.</h3>

In [584]:
fixed = toronto_addresses['Neighbourhood'].apply(', '.join).to_frame()
toronto_addresses1 = pd.concat([toronto_addresses['Postcode'],toronto_addresses['Borough'],fixed,toronto_addresses['Latitude'],toronto_addresses['Longitude']],axis=1)
toronto_addresses1

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
3,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
4,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
5,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
6,M4E,East Toronto,The Beaches,43.676357,-79.293031
7,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
8,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
9,M4M,East Toronto,Studio District,43.659526,-79.340923


In [585]:
!pip install folium
import folium

In [586]:
import requests
from pandas.io.json import json_normalize
client_id = 'ZH3EGCCGTUVO005IHZCHZOYDQN1LQKMKI3B1NREY2HBQKKSZ'
client_secret = 'KOKGU22BQFVB55MVMWQY4QVR2SVCHEADLLM5LWDK0EOWIDJ3'
version = '20180605'

<h3>Filter out the categories of the venues from the JSON file, then filter the columns by creating a function which allocates the venues to a neighbourhood.</h3>

In [587]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


<h3>Render the new dataframe with including category, venue name, etc.</h3>

In [588]:
toronto_venues = getNearbyVenues(names=toronto_addresses1['Neighbourhood'],
                                   latitudes=toronto_addresses1['Latitude'],
                                   longitudes=toronto_addresses1['Longitude']
                                  )

Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea
The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Business Reply Mail Processing Centre 969 Eastern
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Ra

In [670]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Dovercourt Village, Dufferin",43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar
1,"Dovercourt Village, Dufferin",43.669005,-79.442259,Parallel,43.669516,-79.438728,Middle Eastern Restaurant
2,"Dovercourt Village, Dufferin",43.669005,-79.442259,Happy Bakery & Pastries,43.667050,-79.441791,Bakery
3,"Dovercourt Village, Dufferin",43.669005,-79.442259,Planet Fitness Toronto Galleria,43.667588,-79.442574,Gym / Fitness Center
4,"Dovercourt Village, Dufferin",43.669005,-79.442259,FreshCo,43.667918,-79.440754,Supermarket


In [590]:
toronto_venues.shape

(1707, 7)

In [591]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


<h3> We now want to see the distribution of venue categories within each neighbourhood.</h3>

In [592]:
len(toronto_venues['Neighbourhood'].unique())

38

In [593]:
len(toronto_venues['Venue Category'].unique())

238

<h3>Get dummy variables for the neighbourhood samples and their respective venue category. Then, add back the neighbouhood column and set as index. Finally, group the table by the neighbourhood column and the mean of the categories found within them.</h3>

In [594]:
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'])
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']
toronto_onehot.set_index('Neighbourhood',inplace=True)
toronto_onehot

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
Neighbourhood,,,,,,,,,,,,,,,,,,,,,
"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3>Group by 'neighbourhood' and determine the distribution of the categories of venues.</h3>

In [595]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.0,...,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.01,0.0,0.000000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.00,0.0,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.047619
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000


<h3>Create tables which show the most frequently occuring venue categories in each neighbourhood.</h3>

In [596]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4                  Bar  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2   Cheese Shop  0.03
3      Beer Bar  0.03
4    Steakhouse  0.03


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.10
1     Coffee Shop  0.10
2            Café  0.10
3     Yoga Studio  0.05
4             Gym  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1          Comic Shop  0.06
2       Auto Workshop  0.06
3                Park  0.06
4    Recording Studio  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Termin

<h3>Create a function that will return the most common venues in descending order.</h3>

In [597]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h3>Create a table which displays the nth most common venue.</h3>

In [679]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,American Restaurant,Sushi Restaurant,Restaurant,Bakery,Cosmetics Shop
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Farmers Market,Steakhouse,Beer Bar,Seafood Restaurant,Bakery,Cheese Shop,Jazz Club
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Yoga Studio,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Gym / Fitness Center,Spa,Restaurant,Recording Studio,Burrito Place,Skate Park,Auto Workshop,Fast Food Restaurant,Farmers Market
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Plane,Coffee Shop,Sculpture Garden,Boat or Ferry,Boutique


In [644]:
neighbourhood_clustering = toronto_grouped.drop('Neighbourhood', axis=1)
neighbourhood_clustering.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.0,0.0,...,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.01,0.0,0.000000
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.00,0.0,0.000000
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.047619
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
4,0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000


<h3>Create the KMeans model in order to cluster the data.</h3>

In [680]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 4, random_state = 1).fit(neighbourhood_clustering)
kmeans.labels_

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       1, 3, 0, 3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 0, 3, 3], dtype=int32)

In [681]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.drop('Neighborhood', axis=1, inplace=True)

In [682]:
to_merge = toronto_addresses1.sort_values(by='Neighbourhood')
to_merge.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
28,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
26,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
2,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
10,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
34,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


In [683]:
final = pd.concat([to_merge, neighborhoods_venues_sorted], axis=1)
final.sort_values(by='Neighbourhood',inplace=True)
final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,2,Pool,Garden,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
26,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Breakfast Spot,Gift Shop,Italian Restaurant,Dessert Shop,Bank,Bar,Restaurant,Dog Run,Eastern European Restaurant,Cuban Restaurant
2,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,3,Coffee Shop,Breakfast Spot,Café,Yoga Studio,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store,Bar
10,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,3,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Seafood Restaurant,Bakery,Steakhouse,Gym,Deli / Bodega
34,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,3,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Burger Joint,Flower Shop,Indian Restaurant,Pub,Jewish Restaurant


<h3>Plot the venues of the various clusters in matching colors.</h3>

In [684]:
import matplotlib.cm as cm
import matplotlib.colors as colors
kclusters=5

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final['Latitude'], final['Longitude'], final['Neighbourhood'], final['Cluster Labels']):
    label = folium.Popup(poi + ' Cluster ', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Analyze the different clusters to identify the differentiating factor(s).</h3>

In [685]:
final.loc[final['Cluster Labels'] == 0]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Park,Playground,Trail,Building,Yoga Studio,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant
17,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Park,Sushi Restaurant,Home Service,Trail,Jewelry Store,Yoga Studio,Donut Shop,Dive Bar,Dog Run,Doner Restaurant
24,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Playground,Restaurant,Trail,Yoga Studio,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Dumpling Restaurant
35,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0,Health Food Store,Pub,Neighborhood,Other Great Outdoors,Trail,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dumpling Restaurant


In [686]:
print('Cluster 1: ')

Cluster 1: 


In [687]:
final.loc[final['Cluster Labels'] == 1]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M4Y,Downtown Toronto,Church and Wellesley,43.66586,-79.38316,1,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [688]:
final.loc[final['Cluster Labels'] == 2]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,2,Pool,Garden,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [689]:
final.loc[final['Cluster Labels'] == 3]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Breakfast Spot,Gift Shop,Italian Restaurant,Dessert Shop,Bank,Bar,Restaurant,Dog Run,Eastern European Restaurant,Cuban Restaurant
2,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,3,Coffee Shop,Breakfast Spot,Café,Yoga Studio,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store,Bar
10,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,3,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Seafood Restaurant,Bakery,Steakhouse,Gym,Deli / Bodega
34,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,3,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Burger Joint,Flower Shop,Indian Restaurant,Pub,Jewish Restaurant
21,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,3,Bar,Café,Mexican Restaurant,Diner,Music Venue,Italian Restaurant,Bakery,Speakeasy,Fried Chicken Joint,Flea Market
33,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,3,Café,Coffee Shop,Bakery,Gastropub,Italian Restaurant,American Restaurant,Music Store,Coworking Space,Latin American Restaurant,Bookstore
37,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Bookstore,Brewery,Bubble Tea Shop,Restaurant,Pub
31,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,3,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Bakery,Breakfast Spot,Cocktail Bar,Clothing Store,Gastropub
14,M4S,Central Toronto,Davisville,43.704324,-79.388790,3,Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Bar,Deli / Bodega,Gastropub,American Restaurant,Japanese Restaurant
12,M4P,Central Toronto,Davisville North,43.712751,-79.390197,3,Hotel,Gym,Park,Sandwich Place,Breakfast Spot,Clothing Store,Dog Run,Food & Drink Shop,Donut Shop,Dive Bar
